# Jupyter Notebook Para DATA MINING

In [72]:
# Importações
import pandas
import sqlite3
import numpy

# Carregando os dados
data = pandas.read_csv('data/acidentes2023_todas_causas_tipos.csv', encoding='ISO-8859-1', sep=";")
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 514799 entries, 0 to 514798
Data columns (total 37 columns):
 #   Column                  Non-Null Count   Dtype  
---  ------                  --------------   -----  
 0   id                      514799 non-null  int64  
 1   pesid                   514799 non-null  int64  
 2   data_inversa            514799 non-null  object 
 3   dia_semana              514799 non-null  object 
 4   horario                 514799 non-null  object 
 5   uf                      514799 non-null  object 
 6   br                      513037 non-null  float64
 7   km                      513037 non-null  object 
 8   municipio               514799 non-null  object 
 9   causa_principal         514799 non-null  object 
 10  causa_acidente          514799 non-null  object 
 11  ordem_tipo_acidente     514799 non-null  int64  
 12  tipo_acidente           514799 non-null  object 
 13  classificacao_acidente  514799 non-null  object 
 14  fase_dia            

In [73]:
# Subir base suja para SQLite para visualizar e pesquisar
# con = sqlite3.connect('data/acidentes.sqlite')
# data.to_sql(name="dirty_2023_data", con=con, if_exists='replace', index=False)

514799

In [74]:
for column in data.columns:
    unique = data[column].unique()
    unique_len = len(unique)

    if unique_len > 50:
        print(column, unique_len)
        continue

    print(column, unique_len, unique)

id 67724
pesid 166836
data_inversa 365
dia_semana 7 ['domingo' 'segunda-feira' 'terça-feira' 'quarta-feira' 'quinta-feira'
 'sexta-feira' 'sábado']
horario 1406
uf 27 ['MG' 'RS' 'SE' 'PR' 'SP' 'ES' 'SC' 'RJ' 'MT' 'PE' 'PA' 'CE' 'AM' 'BA'
 'DF' 'RO' 'RN' 'GO' 'PB' 'AL' 'MS' 'PI' 'TO' 'MA' 'AP' 'AC' 'RR']
br 119
km 7638
municipio 1825
causa_principal 2 ['Sim' 'Não']
causa_acidente 76
ordem_tipo_acidente 12 [ 2  1  4  3  5  6  7  9  8 10 11 12]
tipo_acidente 16 ['Queda de ocupante de veículo' 'Tombamento' 'Capotamento'
 'Saída de leito carroçável' 'Colisão com objeto' 'Colisão traseira'
 'Atropelamento de Pedestre' 'Colisão transversal'
 'Atropelamento de Animal' 'Colisão frontal'
 'Colisão lateral mesmo sentido' 'Colisão lateral sentido oposto'
 'Derramamento de carga' 'Engavetamento' 'Incêndio' 'Eventos atípicos']
classificacao_acidente 3 ['Com Vítimas Feridas' 'Com Vítimas Fatais' 'Sem Vítimas']
fase_dia 4 ['Plena Noite' 'Pleno dia' 'Amanhecer' 'Anoitecer']
sentido_via 3 ['Crescente' '

In [75]:
data.isnull().sum()

id                            0
pesid                         0
data_inversa                  0
dia_semana                    0
horario                       0
uf                            0
br                         1762
km                         1762
municipio                     0
causa_principal               0
causa_acidente                0
ordem_tipo_acidente           0
tipo_acidente                 0
classificacao_acidente        0
fase_dia                      0
sentido_via                   0
condicao_metereologica        0
tipo_pista                    0
tracado_via                   0
uso_solo                      0
id_veiculo                    0
tipo_veiculo                  0
marca                     21014
ano_fabricacao_veiculo    27439
tipo_envolvido                0
estado_fisico                 0
idade                     49369
sexo                          0
ilesos                        0
feridos_leves                 0
feridos_graves                0
mortos  

In [76]:
print('Number of duplicate rows = %d' % (data.duplicated().sum()))

Number of duplicate rows = 0


# Conversão de dados

In [77]:
# Dia da semana
week_2_number = {
    'segunda-feira': 0,
    'terça-feira': 1,
    'quarta-feira': 2,
    'quinta-feira': 3,
    'sexta-feira': 4,
    'sábado': 5,
    'domingo': 6,
}

for key in week_2_number.keys():
    data['dia_semana'].replace(key, week_2_number[key], inplace=True)

data['dia_semana'].unique()

/tmp/ipykernel_65838/1895330286.py:13: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  data['dia_semana'].replace(key, week_2_number[key], inplace=True)
/tmp/ipykernel_65838/1895330286.py:13: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  data['dia_semana'].replace(key, week_2_number[key], inplace=True

array([6, 0, 1, 2, 3, 4, 5])

In [78]:
data_hora = []
for i in range(len(data)):
    data_hora.append(data['data_inversa'][i] + ' ' + data['horario'][i][:5] + 'Z')

data['data_hora'] = pandas.to_datetime(data_hora, format='ISO8601')


### Cast para str

In [79]:
fields_2_str = ['data_inversa', 'horario', 'uf', 'municipio', 'causa_acidente', 'tipo_acidente', 'classificacao_acidente', 'fase_dia', 'sentido_via', 'condicao_metereologica', 'tipo_pista', 'tracado_via', 'tipo_veiculo', 'marca', 'tipo_envolvido', 'estado_fisico', 'sexo', 'latitude', 'longitude', 'regional', 'delegacia', 'uop']
for field in fields_2_str:
    data[field] = data[field].convert_dtypes('string')


### Cast Boolean

In [80]:
data['causa_principal'] = data['causa_principal'].replace(['Sim', 'sim'], True)
data['causa_principal'] = data['causa_principal'].replace(['Não', 'não'], False)

data['uso_solo'] = data['uso_solo'].replace(['Sim', 'sim'], True)
data['uso_solo'] = data['uso_solo'].replace(['Não', 'não'], False)

/tmp/ipykernel_65838/1541778264.py:2: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  data['causa_principal'] = data['causa_principal'].replace(['Não', 'não'], False)
/tmp/ipykernel_65838/1541778264.py:5: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  data['uso_solo'] = data['uso_solo'].replace(['Não', 'não'], False)


### Cast Float

In [85]:
km_to_float = []
for value in data['km']:
    tmp = value
    if isinstance(value, str):
        tmp = value.replace(',', '.')

    km_to_float.append(float(tmp))

data['km'] = km_to_float

### Reorder

In [82]:
data.columns.tolist()
# data = data[cols]
cols = ['id', 'pesid', 'data_hora', 'data_inversa', 'dia_semana', 'horario', 'uf', 'br', 'km', 'municipio', 'causa_principal', 'causa_acidente', 'ordem_tipo_acidente', 'tipo_acidente', 'classificacao_acidente', 'fase_dia', 'sentido_via', 'condicao_metereologica', 'tipo_pista', 'tracado_via', 'uso_solo', 'id_veiculo', 'tipo_veiculo', 'marca', 'ano_fabricacao_veiculo', 'tipo_envolvido', 'estado_fisico', 'idade', 'sexo', 'ilesos', 'feridos_leves', 'feridos_graves', 'mortos', 'latitude', 'longitude', 'regional', 'delegacia', 'uop',]
data = data[cols]

# Pós processamento

In [86]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 514799 entries, 0 to 514798
Data columns (total 38 columns):
 #   Column                  Non-Null Count   Dtype              
---  ------                  --------------   -----              
 0   id                      514799 non-null  int64              
 1   pesid                   514799 non-null  int64              
 2   data_hora               514799 non-null  datetime64[ns, UTC]
 3   data_inversa            514799 non-null  string             
 4   dia_semana              514799 non-null  int64              
 5   horario                 514799 non-null  string             
 6   uf                      514799 non-null  string             
 7   br                      513037 non-null  float64            
 8   km                      513037 non-null  float64            
 9   municipio               514799 non-null  string             
 10  causa_principal         514799 non-null  bool               
 11  causa_acidente          51

In [87]:
data[:10]

,id,pesid,data_hora,data_inversa,dia_semana,horario,uf,br,km,municipio,...,sexo,ilesos,feridos_leves,feridos_graves,mortos,latitude,longitude,regional,delegacia,uop
0,496506,1082142,2023-01-01 00:15:00+00:00,2023-01-01,6,00:15:00,MG,116.0,587.0,MANHUACU,...,Masculino,0,1,0,0,"-20,24173903","-42,15868042",SPRF-MG,DEL06-MG,UOP03-DEL06-MG
1,496506,1082142,2023-01-01 00:15:00+00:00,2023-01-01,6,00:15:00,MG,116.0,587.0,MANHUACU,...,Masculino,0,1,0,0,"-20,24173903","-42,15868042",SPRF-MG,DEL06-MG,UOP03-DEL06-MG
2,496506,1082142,2023-01-01 00:15:00+00:00,2023-01-01,6,00:15:00,MG,116.0,587.0,MANHUACU,...,Masculino,0,1,0,0,"-20,24173903","-42,15868042",SPRF-MG,DEL06-MG,UOP03-DEL06-MG
3,496506,1082142,2023-01-01 00:15:00+00:00,2023-01-01,6,00:15:00,MG,116.0,587.0,MANHUACU,...,Masculino,0,1,0,0,"-20,24173903","-42,15868042",SPRF-MG,DEL06-MG,UOP03-DEL06-MG
4,496507,1082138,2023-01-01 00:20:00+00:00,2023-01-01,6,00:20:00,MG,381.0,686.5,LAVRAS,...,Masculino,0,1,0,0,"-21,17928973","-45,12256622",SPRF-MG,DEL04-MG,UOP02-DEL04-MG
5,496512,1082880,2023-01-01 00:55:00+00:00,2023-01-01,6,00:55:00,RS,386.0,353.2,ESTRELA,...,Feminino,0,0,1,0,"-29,50160055","-51,94784403",SPRF-RS,DEL04-RS,UOP02-DEL04-RS
6,496512,1082880,2023-01-01 00:55:00+00:00,2023-01-01,6,00:55:00,RS,386.0,353.2,ESTRELA,...,Feminino,0,0,1,0,"-29,50160055","-51,94784403",SPRF-RS,DEL04-RS,UOP02-DEL04-RS
7,496512,1082880,2023-01-01 00:55:00+00:00,2023-01-01,6,00:55:00,RS,386.0,353.2,ESTRELA,...,Feminino,0,0,1,0,"-29,50160055","-51,94784403",SPRF-RS,DEL04-RS,UOP02-DEL04-RS
8,496512,1082880,2023-01-01 00:55:00+00:00,2023-01-01,6,00:55:00,RS,386.0,353.2,ESTRELA,...,Feminino,0,0,1,0,"-29,50160055","-51,94784403",SPRF-RS,DEL04-RS,UOP02-DEL04-RS
9,496512,1082880,2023-01-01 00:55:00+00:00,2023-01-01,6,00:55:00,RS,386.0,353.2,ESTRELA,...,Feminino,0,0,1,0,"-29,50160055","-51,94784403",SPRF-RS,DEL04-RS,UOP02-DEL04-RS
